<a href="https://colab.research.google.com/github/Plogeur/HAI923/blob/master/Notebook_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
# To generate GIFs
!pip install imageio
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-g2u63iau
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-g2u63iau
  Resolved https://github.com/tensorflow/docs to commit 70174a87eb9c39b6f71d948801e140e9436e66e8
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2023.12.6.69331-py3-none-any.whl size=181263 sha256=3d665814a035f12e36453816629dfc727ead311f19968aa877204c4c9c1657f4
  Stored in directory: /tmp/pip-ephem-wheel-cache-kskbylie/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


In [ ]:
import os
import random
import shutil
import keras
import imageio
import glob
import pathlib
import sys
import cv2
import numpy as np
import seaborn as sns
import pandas as pd
from numpy import mean
from numpy import std
from keras import backend as K
from IPython import display
import matplotlib.cm as cm
import tensorflow as tf
from numpy import ones, zeros
from keras import metrics
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from numpy.random import randint
from tensorflow.keras.regularizers import L1L2
import tensorflow_docs.vis.embed as embed
from tensorflow.keras.models import Sequential
from tensorflow.keras.saving import load_model
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, Adamax, Lion
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import Input, Activation, Dropout, Dense, GlobalAveragePooling2D, Reshape, Flatten, Rescaling, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, LeakyReLU

In [ ]:
# GLOBAL VARIABLE
IMG_SIZE = 124
BATCH_SIZE = 4
CHANEL = 3
IMG_SHAPE=(IMG_SIZE, IMG_SIZE, CHANEL)
EPOCHS = 1000
VERBOSE = 1
COLUMNS = 25
SEED = 123
PATH_DIR = "/content/gdrive/MyDrive/Colab_Notebooks/"
POLICE_SIZE = 18 # Taille de la police pour les plot
plt.rcParams.update({'font.size': POLICE_SIZE})

# SET SEED
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/MyDrive/Colab_Notebooks/
path = %pwd
%mkdir Résultats/GAN
%mkdir Résultats/checkpoint

Mounted at /content/gdrive/
[Errno 2] No such file or directory: 'gdrive/MyDrive/Colab_Notebooks/'
/content
mkdir: cannot create directory ‘Résultats/GAN’: No such file or directory
mkdir: cannot create directory ‘Résultats/checkpoint’: No such file or directory


# Model

## GAN

In [ ]:
def create_generator(latent_dim=100):
  model = Sequential()
    # 1/4 de l'image (7x7) * nombre d'images possibles
  #n_nodes = 124 * 31 * 31
  n_nodes = 100 * 31 * 31
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  #model.add(Reshape((31, 31, 124)))
  model.add(Reshape((31, 31, 100)))
  # upsample to 14x14
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  # upsample to 28x28
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(3, (7,7), activation='sigmoid', padding='same'))
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
def create_discriminator():
  model = Sequential()
  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=[124,124,3]))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.3))
  model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.3))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
def create_training_data(path_data, list_classes, gray=False):
  training_data=[]
  for classes in list_classes:
    path=os.path.join(path_data, classes)
    class_num=list_classes.index(classes)
    for img in os.listdir(path):
      try :
        if gray == False :
          img_array = cv2.imread(os.path.join(path,img), cv2.COLOR_BGR2RGB)
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        else :
          img_array = cv2.imread(os.path.join(path,img), cv2.COLOR_BGR2GRAY)
          equ = cv2.equalizeHist(new_array)
          new_array = np.hstack((new_array,equ))
        training_data.append([new_array, class_num])
      except Exception as e:
        pass
  return training_data

def create_X_y(path_data, list_classes, gray=False, onehot=False):
      training_data=create_training_data(path_data, list_classes, gray)
      random.shuffle(training_data)
      X=[]
      y=[]
      for features, label in training_data:
        X.append(features)
        y.append(label)
      if gray == False :
        X=np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 3)
      else :
        X=np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE, 1)
      if onehot == False :
        y=np.array(y, dtype=np.float32)
      return X,y

# Chargement et normalisation des données
def load_data(my_path,animal):
  my_classes=[animal]
  X,y=create_X_y(my_path,my_classes)
  # Surtout ne pas oublier de normaliser les données avec :
  X=X.astype('float32')
  X=X/255.0
  return X

# Creation d'un jeu de données de vraies images
# les vraies images sont labélisées avec 1
def generate_real_samples(dataset, nb_images):
  # tirage aléatoire
  ix = randint(0, dataset.shape[0], nb_images)
  #serie = random.sample(range(0, dataset.shape[0]), nb_images)
  # sélection des images
  X = dataset[ix]
  # mettre 1 comme label de classe
  y = ones((nb_images, 1))
  return X, y

# Création d'un faux jeu de données
# elles seront labélisées avec 0
def generate_fake_samples(nb_images):
  X = np.random.rand(IMG_SIZE * IMG_SIZE * 3 * nb_images)
  # reshape en images grises
  X = X.reshape((nb_images, IMG_SIZE, IMG_SIZE, 3))
  # mettre 0 comme label de classe
  y = zeros((nb_images, 1))
  return X, y

In [ ]:
# train the discriminator model
def train_discriminator(model, dataset, epochs=100, batchsize=20):
  # on constitue un jeu de données de batchsize/2 images réelles et images fausses
	half_batch = int(batchsize / 2)
	# boucler sur les epochs
	for i in range(epochs):
		# sélection d'images réelles
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# mettre à jour le discriminateur avec les images réelles
		_, real_acc = model.train_on_batch(X_real, y_real)
		# generation de fausses images
		X_fake, y_fake = generate_fake_samples(half_batch)
		# mise à jour du discriminateur avec de fausses images
		_, fake_acc = model.train_on_batch(X_fake, y_fake)

In [ ]:
#@tf.function # This annotation causes the function to be "compiled"
def create_gan(generator_model, discriminator_model):
  # mettre les poids du discriminateur non entrable
  discriminator_model.trainable = False
  # création d'un seul modele qui regroupe generateur et discriminateur
  model = Sequential()
  # ajout du générateur
  model.add(generator_model)
  # ajout du discriminateur
  model.add(discriminator_model)
  opt = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy', optimizer=opt)
  return model

In [ ]:
def generate_latent_points(latent_dim, nb_images):
  X_input = np.random.randn(latent_dim * nb_images)
  X_input = X_input.reshape(nb_images, latent_dim)
  return X_input

def generate_fake_samples_for_generator(generator_model, latent_dim, nb_images):
  # generation des points
  X_input = generate_latent_points(latent_dim,nb_images)
  # prediction de la sortie du générateur
  X = generator_model.predict(X_input, verbose=0)
  # A ce niveau on considère que les images sont fausses
  # donc on met 0 comme label.
  y = zeros((nb_images, 1))
  return X, y

def evaluate_model(dataset, epoch, generator_model, discriminator_model, latent_dim, nb_images=100, save_model=False):
  # récupération de vraies images pour le discriminateur
  X_real, y_real = generate_real_samples(dataset, nb_images)
  # Evaluation de l'accuracy pour le discriminateur
  _, acc_real = discriminator_model.evaluate(X_real, y_real, verbose=0)

  # génération de fausses images pour le générateur et donc le gan
  X_fake, y_fake = generate_fake_samples_for_generator(generator_model, latent_dim, nb_images)
  # Evaluation du discriminateur avec des fausses images
  _, acc_fake = discriminator_model.evaluate(X_fake, y_fake, verbose=0)

  # Save model each 100 epochs
  if save_model==True and epoch % 100 == 0 :
    # sauvegarde du generateur pour un autre usage
    print('save checkpoint')
    filename = '/content/gdrive/MyDrive/Colab Notebooks/Résultats/checkpoint/generator_model_GAN%03d.h5' % (epoch)
    generator_model.save(filename)

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=25):
  # Pour déterminer combien il y aura de batchs analysés à chaque epoch
  batch_per_epoch = int(dataset.shape[0] / n_batch)
  half_batch = int(n_batch / 2)
	# manually enumerate epochs
  for epoch in range(1, n_epochs) :
		# enumerate batches over the training set
    print("epoch n°", epoch)
    for batch in range(batch_per_epoch) :
			  # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch(X_real, y_real, reset_metrics=False)
        # generate 'fake' examples
        X_fake, y_fake = generate_fake_samples_for_generator(g_model, latent_dim, half_batch)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch(X_fake, y_fake, reset_metrics=False)
        # prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan, reset_metrics=False)

    if epoch % 10 == 0 :
      # Produce images for the GIF
      display.clear_output(wait=True)

      evaluate_model(dataset, epoch, generator_model, discriminator_model, latent_dim, 100, True)
      latent_dim=100
      COLUMNS = 9
      plt.figure(figsize=(5,5))

      # Add an epoch counter to the top middle of the plot
      plt.text(0.5, 0.95, f'Epoch: {epoch}', horizontalalignment='center', verticalalignment='center',
                transform=plt.gca().transAxes, fontsize=10, color='white')

      for image in range(COLUMNS):
        plt.subplot(3,3,image+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        noise=tf.random.normal([1,latent_dim])
        plt.imshow((generator_model.predict(noise, verbose=0)[0]+1)/2)
      print(epoch)
      plt.savefig('/content/gdrive/MyDrive/Colab Notebooks/Résultats/GAN/image_at_epoch_{:04d}.png'.format(epoch), bbox_inches='tight')
      plt.show()

## Train

In [ ]:
#S'il y a un model enregistré :
#generator_model = load_model('/content/gdrive/MyDrive/generator_model_GAN400.h5')

#Sinon création du générateur
generator_model = create_generator()

In [ ]:
#!wget https://www.lirmm.fr/~poncelet/Ressources/Tiger-Fox-Elephant.zip

#import zipfile
#with zipfile.ZipFile("Tiger-Fox-Elephant.zip","r") as zip_ref:
    #zip_ref.extractall("/content/gdrive/MyDrive/Colab Notebooks/Data_Project")

ARCHI 1


In [ ]:
latent_dim=100
epochs=10000
batchsize=32
CHANEL=3
# Création du discriminateur
discriminator_model = create_discriminator()

# Création du GAN
gan_model = create_gan(generator_model, discriminator_model)
# Chargement des données
dataset = load_data('/content/gdrive/MyDrive/Colab Notebooks/Data_Projet/Tiger-Fox-Elephant/', "fox")

# train model
train(generator_model, discriminator_model, gan_model, dataset, latent_dim, epochs, batchsize)

epoch n° 1


## GIF

In [ ]:
anim_file = 'Gan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('Résultats/GAN/image*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.v2.imread(filename)
    writer.append_data(image)
  image = imageio.v2.imread(filename)
  writer.append_data(image)

embed.embed_file(anim_file)

%rm Résultats/GAN/*.png